# Visualization of current COVID-19 trend in Shanghai

The visualization method is based on MinutePhysics' video [here](https://www.youtube.com/watch?v=54XLXg4fYsc).

Data depends on the work of [@kekincai](https://github.com/kekincai), downloaded from the deployed app available [here](https://kapaul.shinyapps.io/shanghai_covid19/).

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('全体数据一览.csv').drop_duplicates(subset=['date']).sort_values('date')
dataset['新增本土确诊含无症状'] = dataset['新增本土新冠肺炎确诊病例'] + dataset['新增本土无症状感染者']
dataset.head()

,Unnamed: 0,date,新增本土新冠肺炎确诊病例,新增本土无症状感染者,新增境外输入性新冠肺炎确诊病例,新增境外输入性无症状感染者,治愈出院,解除医学观察无症状感染者,新增本土确诊含无症状,无症状闭环隔离管控人数,无症状风险人群筛查人数,非管控区域病例比例
41,23,2022-03-01,1,1,37,17,0,0,2,0,1,100.0
40,22,2022-03-03,2,14,43,21,0,0,16,0,14,100.0
39,21,2022-03-04,3,16,24,10,0,0,19,0,16,100.0
38,20,2022-03-05,0,28,25,10,0,0,28,0,28,100.0
37,191,2022-03-06,3,45,32,16,0,0,48,0,45,100.0


## Calculate our two axes

MinutePhysics recommeded using a moving average to counter the random fluctuations in daily new cases, so that's what we do here. We calculate a rolling average with a 3-day window.

The new cases are plotted not against **time**, but against **the total number of cases**. So we calculate the cumulated sum as well.

In [11]:
new_cases = dataset['新增本土确诊含无症状'].rolling(5).sum()
new_cases.describe()

count        36.000000
mean      28469.444444
std       39593.859964
min         113.000000
25%         518.250000
50%        7628.000000
75%       35623.250000
max      124326.000000
Name: 新增本土确诊含无症状, dtype: float64

In [12]:
all_cases = dataset['新增本土确诊含无症状'].cumsum()
all_cases.describe()

count        40.000000
mean      44023.875000
std       70424.249171
min           2.000000
25%         519.750000
50%        6129.500000
75%       52781.500000
max      255497.000000
Name: 新增本土确诊含无症状, dtype: float64

In [13]:
vis = pd.DataFrame({
    'Day': dataset['date'],
    'New Cases': new_cases,
    'All Cases': all_cases
}).dropna()

vis.head()

,Day,New Cases,All Cases
37,2022-03-06,113.0,113
36,2022-03-07,166.0,168
35,2022-03-08,215.0,233
34,2022-03-09,276.0,313
33,2022-03-10,323.0,388


## Create the plot!

Here we use Plotly's built-in animation feature. Note that, in accordance to MinutePhysics' video, we're using logarithmic scales here for both X and Y axis.

I've added in a reference line (blue), as well as the overall trace (red).

In [31]:
import plotly.express as px
import plotly.graph_objects as go

X_AXIS = "All Cases"
Y_AXIS = "New Cases"

fig = px.scatter(vis, x=X_AXIS, y=Y_AXIS, 
                 animation_frame="Day",
                 range_x=[1, vis[X_AXIS].max() * 2], 
                 range_y=[1, vis[Y_AXIS].max() * 2], 
                 log_x=True, log_y=True,
                 color_discrete_sequence=['red'])

fig.update_layout(shapes = [{
    'type': 'line', 
    'yref': 'paper', 
    'xref': 'paper', 
    'y0': 0, 'y1': 1, 
    'x0': 0, 'x1': 1,
    'line': {
        'color': 'rgba(100, 100, 255, 0.5)'
    }
}])
fig.add_trace(go.Scatter(
    x=vis['All Cases'], 
    y=vis['New Cases'], 
    text=vis['Day'],
    mode='markers+lines+text',
    textposition="top left",
    textfont=dict(size=4),
    name="Shanghai"
))
fig.write_html("index.html")
fig.show()